In [67]:
import pandas as pd
import itertools
import numpy as np
import openpyxl

In [68]:
fuel_dict = {
    1: (0.178, 0.094, 'mgo'),      # MGO
    2: (0.146, 0.073, 'lng'),      # LNG
    3: (0.491, 0.261, 'methanol'), # GREEN METHANOL
    4: (0.324, 0.178, 'nh3'), # GREEN NH3
    5: (0.278, 0.147, 'B30'),         # B30
    6: (0.245, 0.130, 'B20'),         # B20
    7: (0.228, 0.121, 'B15'),         # B15
    8: (0.201, 0.106, 'B7'),         # B7
    9: (0.227, 0.108, 'EL40')  # EL40
}

t2_values = range(2,6)
w_values = range(1,7)
scenario_range = [1,2,3,4,5,6]
period_range = [2, 3, 4, 5]
# r_values = range(1,1819)

fuel_price_csv = 'fuel_price2_6s.csv'



In [69]:
scenario_developments = [
    {'fossil': [-0.026, -0.026, -0.026, -0.026], 'bio': [0.12, 0.12, 0.12,0.12], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}, # high, high, high 
    {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [0.12, 0.12, 0.12, 0.12], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}, #LHH
    {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [-0.051, -0.051, -0.051, -0.051], 'green_fuels': [-0.073, -0.073, -0.073, -0.073]}, 
    {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [0.038, 0.038, 0.038, 0.038], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]},
    {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [-0.051, -0.051, -0.051, -0.051], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]},
    {'fossil': [-0.069, -0.069, -0.069, -0.069], 'bio': [0.038, 0.038, 0.038, 0.038], 'green_fuels': [-0.073, -0.073, -0.073, -0.073]}
]


In [70]:
def generate_fuel_prices(scenario_range, period_range, fuel_dict, scenario_developments):
    num_scenarios = len(scenario_range)
    num_periods = len(period_range)
    num_fuels = len(fuel_dict)
    
    fuel_prices = np.zeros((num_fuels, num_periods, num_scenarios))
    
    group_scaling_factors = {}
    
    for _, (upper_bound, lower_bound, group) in fuel_dict.items():
        group_scaling_factors[group] = np.ones((num_scenarios, num_periods))  # Initialize scaling factors to ones
    
    # Apply scenario developments
    for s_idx, scenario_dev in enumerate(scenario_developments):
        print('s_idx ', s_idx, 'scenario_dev', scenario_dev)
        for group, developments in scenario_dev.items():
            if group in group_scaling_factors:
                for p_idx, period_dev in enumerate(developments):
                    #print('p_idx: ', p_idx, 'period_dev ', period_dev)
                    if p_idx >= 1:  # Start applying incremental developments from the third period onwards
                        group_scaling_factors[group][s_idx, p_idx:] += period_dev
    
    for i, (fuel_key, (upper_bound, lower_bound, group)) in enumerate(fuel_dict.items()):
        base_price = lower_bound + (upper_bound - lower_bound)
        for s_idx, scenario in enumerate(scenario_range):
            for p_idx, period in enumerate(period_range):
                #print(group_scaling_factors[group][s_idx, p_idx])
                fuel_price = base_price * group_scaling_factors[group][s_idx, p_idx]
                fuel_prices[fuel_key - 1, p_idx, s_idx] = fuel_price  # Adjusted for zero-based indexing
    
    return fuel_prices


In [71]:
fuel_prices = generate_fuel_prices(w_values, t2_values, fuel_dict, scenario_developments)
print(fuel_prices)

s_idx  0 scenario_dev {'fossil': [-0.026, -0.026, -0.026, -0.026], 'bio': [0.12, 0.12, 0.12, 0.12], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}
s_idx  1 scenario_dev {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [0.12, 0.12, 0.12, 0.12], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}
s_idx  2 scenario_dev {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [-0.051, -0.051, -0.051, -0.051], 'green_fuels': [-0.073, -0.073, -0.073, -0.073]}
s_idx  3 scenario_dev {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [0.038, 0.038, 0.038, 0.038], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}
s_idx  4 scenario_dev {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [-0.051, -0.051, -0.051, -0.051], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}
s_idx  5 scenario_dev {'fossil': [-0.069, -0.069, -0.069, -0.069], 'bio': [0.038, 0.038, 0.038, 0.038], 'green_fuels': [-0.073, -0.073, -0.073, -0.073]}
[[[0.178    0.178    0.178    0.178    0.178    0.178   ]
  [0.173372 0.157886 0.1

In [72]:
fuel_prices_data = []

for fuel_key, (upper_bound, lower_bound, _) in fuel_dict.items():
    fuel_idx = list(fuel_dict.keys()).index(fuel_key)
    for s_idx, scenario in enumerate(w_values):
        for p_idx, period in enumerate(t2_values):
            
                fuel_prices_data.append({
                    'F': fuel_key,
                    #'R': route,
                    'T': period,
                    'S': scenario,
                    'Value': fuel_prices[fuel_idx, p_idx, s_idx]
                })

# Create DataFrame from the list of dictionaries
fuel_prices_df = pd.DataFrame(fuel_prices_data)

# for route in r_values:
#                 
fuel_prices_df.to_csv(fuel_price_csv, index=False)

print(fuel_prices_df)

     F  T  S     Value
0    1  2  1  0.178000
1    1  3  1  0.173372
2    1  4  1  0.168744
3    1  5  1  0.164116
4    1  2  2  0.178000
..  .. .. ..       ...
211  9  5  5  0.205889
212  9  2  6  0.227000
213  9  3  6  0.210429
214  9  4  6  0.193858
215  9  5  6  0.177287

[216 rows x 4 columns]
